# Case Study 2: Modeling (part 2)


In [ ]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.append("../python")

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

In [ ]:
# Load dataset

from mll1 import attrition

df_attr = attrition.load_dataset(
    "../../data/employee_attrition/Employee_attrition.xlsx"
)
df_attr

## Model selection and hyper-parameters optimization

The same strategies for model and hyper-parameters selection as we have seen for regression are usable with classification.

The main differences will consist in:

* the way subsets are built;
* the score used (default is accuracy in scikit-learn).


### Stratified cross validation

With classification, especially if the classes are imbalanced, it can be important to ensure that the proportions of the classes are the same in the train set and the test set.

This is the purpose of **Stratified Cross Validation**. Stratified means here that each class is sampled independently.

Let's look at the repartition between positive and negative classes:

In [ ]:
_ = sns.catplot(data=df_attr, x="has_left", kind="count")

We see that the classes are imbalanced (not severely). This has two consequences:

* We will need to perform a stratified cross-validation
* The accuracy score may not be appropriate.

### Selecting the best model

We will now unfold the model selection and hyper-parameter optimization procedure.


#### Stratified train test split

First we define our train and test sets with a stratified approach:

In [ ]:
from sklearn.model_selection import train_test_split

X = df_attr.drop(["Emp ID", "has_left"], axis=1)
y = df_attr.has_left.values

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, stratify=y, random_state=0)

Note that the stratification need not be on the target value only.

#### Defining the models

We will basically test 3 model families and for each one optimize their hyper-parameters.
The models are:
- `LogisticRegression`
- `RandomForestClassifier`
- `HistGradientBoostingClassifier`.

In the code below we define the three models along their preprocessing steps:

In [ ]:
from sklearn.pipeline import Pipeline
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier

model_logist = Pipeline(
    [
        ("cat_enc", TargetEncoder(cols=["dept", "salary"])),
        ("scaler", StandardScaler()),
        ("poly", PolynomialFeatures()),
        ("clf", LogisticRegression(max_iter=1000)),
    ]
)

model_rf = Pipeline(
    [
        ("cat_enc", TargetEncoder(cols=["dept", "salary"])),
        ("clf", RandomForestClassifier(random_state=1)),
    ]
)

model_gb = Pipeline(
    [
        ("cat_enc", TargetEncoder(cols=["dept", "salary"])),
        ("clf", HistGradientBoostingClassifier(random_state=1)),
    ]
)


#### The parameter disributions

In order to perform a randomized search for the hyper parameters, we need to define the distributions of the parameters for each model:


In [ ]:
from scipy import stats

param_distribs_logist = {
    "cat_enc__min_samples_leaf": np.arange(5, 101),
    "poly__degree": np.arange(1, 6),
    "clf__C": stats.loguniform(1e-2, 1e2),
}

param_distribs_rf = {
    "cat_enc__min_samples_leaf": np.arange(5, 101),
    "clf__min_samples_leaf": np.arange(5, 21),
    "clf__n_estimators": np.arange(10, 500),
}

param_distribs_gb = {
    "cat_enc__min_samples_leaf": np.arange(5, 101),
    "clf__min_samples_leaf": np.arange(5, 21),
    "clf__max_iter": np.arange(10, 500),
    "clf__learning_rate": stats.loguniform(1e-3, 1),
}

#### Stratified K-Fold CV

As for the train / test split, the cross-validation must be performed in a stratified way, in order to maintain the classes proportions in each fold:

In [ ]:
from sklearn.model_selection import StratifiedKFold

cv_strat = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)

Internally this object will be called using its `split(X, y)` method, ensuring the target will be known so the class proportions can be preserved.

It returns an iterator on pairs of `(itrain, itest)` index tables.

**Exercise**:

1. Ensure that the proportion of positive samples are indeed preserved in each split
2. Compare with the result of a non-stratified K-Fold (use the `sklearn.model_selection.KFold` object)

The output should look like:

```
fold: 0; Prop. of pos. in train: 0.2381; in test: 0.2381
fold: 1; Prop. of pos. in train: 0.2380; in test: 0.2380
fold: 2; Prop. of pos. in train: 0.2381; in test: 0.2381
```

Since the classes are not severely imbalanced, and the splits are large (about 5000 samples each) the result should not be too different.

#### The performance measure

We want to perform the cross validation using the **average precision** score. For this we need to specify to the `RandomizedSearchCV` object which scoring function to use.

However there are some difficulties:

* For some performance measures (loss functions for example) lower means better, while for others (the usual score functions) higher is better;

* Some are based on the predicted classes, and others are based on the decision function or the predicted class probabilities.

To make things unambiguous, Scikit-learn allows us to define a `Scorer` object which will take care of the specificities of the performance function:

In [ ]:
from sklearn.metrics import make_scorer, average_precision_score

avg_prec_scorer = make_scorer(
    average_precision_score, greater_is_better=True, needs_proba=True
)

#### Optimizing the hyper-parameters

**Exercise**:

For each model perform a randomized search to optimize the hyper parameters using:
* `<base_model_name>_cv` as the variable name
* the appropriate parameter distributions defined above
* the stratified cross validation object defined above
* the scorer function defined above
* 10 iterations
* use 1 as a random seed
* parallelize the computations with `n_jobs=1` (use as many tasks as you have CPU cores).

Once the model is trained:

* Print the best found parameters
* Plot the result using the `plot_classification_result()` function from the `mll1.plotting` module

**Note**: This can take a long time (several minutes per model)

Let's superimpose the precision-recall curves to compare the models:

In [ ]:
from mll1.plotting import plot_precision_recall_curve

In [ ]:
plot_precision_recall_curve(model_logist_cv, Xtest, ytest)
plot_precision_recall_curve(model_rf_cv, Xtest, ytest, ax=plt.gca())
_ = plot_precision_recall_curve(model_gb_cv, Xtest, ytest, ax=plt.gca())

We seem to have two models which are very close to each other. Is the difference significant ?

To determine that, we can use the cross-validation standard deviation of the scores to estimate the score certainty:

In [ ]:
print(
    f'RF: {avg_prec_scorer(model_rf_cv, Xtest, ytest):0.3f}+-{model_rf_cv.cv_results_["std_test_score"][model_rf_cv.best_index_]:0.3f}'
)
print(
    f'GB: {avg_prec_scorer(model_gb_cv, Xtest, ytest):0.3f}+-{model_gb_cv.cv_results_["std_test_score"][model_gb_cv.best_index_]:0.3f}'
)

The difference does not seem to be significant so we may want to choose on another criterion like the evaluation speed:

In [ ]:
%timeit model_rf_cv.predict(Xtest)
%timeit model_gb_cv.predict(Xtest)

The Random Forest model seems to be the fastest one. It would probably be our best choice.

## Validation strategies and deliverable

The process we have described above is the usual one when the model is to be used later on never-seen-before data points.

**Exercise**:

* Can you explain why it is not adapted here ?
* What would you suggest ?

### Nested cross validation

In the previous use case we've seen different strategies to choose the best hyper-parameter set. Actually those strategies can be used anytime we need to evaluate a model performance.

Since the beginning, we have evaluated the final model performances using a simple train / test (or dev / test) strategy. But any of the evaluation strategies can be used.

We thus have two stages for which we need to choose a performance evaluation strategy:

* the final performance evaluation
* the hyper-parameter optimization (performance is compared between all parameter sets)

And any strategy can be used for each one.

In the above examples, we have typically used:

* A train (dev)/ test strategy for the final performance evaluation
* A K-Fold cross validation strategy for the hyper-parameters optimization.

This however does not allow us to have a prediction for each point of the dataset. For this, we can use a K-Fold cross validation for the final performance evaluation.

This leads to a **nested cross validation** strategy. Using scikit-learn it is quite easy to perform:

In [ ]:
from sklearn.model_selection import cross_val_predict

yscore = cross_val_predict(model_rf_cv, X, y, cv=cv_strat, method="predict_proba").T[1]

The first level of cross-validation here (for the train/test splits) comes from the `cv` parameter given to `cross_val_predict()`, and the second one (for hyper-parameters optimization) from the `cv` parameter that was given to the RandomizedSearchCV constructor when we constructed the model_rf_cv object.

Let examine the score distribution:

In [ ]:
sns.histplot(x=yscore, hue=y, log_scale=(False, True), alpha=0.6)
_ = plt.xlabel("Attrition score")

There is some overlap, but averall the lowest scores corrrespond to the negative samples and the highest to the positive ones (the employees that left the company). Moreover, the second part (high scores) give little false positives.

### Saving scores, models and meta-information

In the next notebook, we will analyze the response of the model for some selected samples. For this we need to be able to determine:

* The sample score
* The fold index of the sample
* The model used to score the sample

since we have one model per fold.

To be able to achieve this, we perform the scoring loops of cross-validation "manually" to be able to retrieve:

* The score values in a `yscore` variable
* The fold index of each sample in a `fold_index` variable
* The list of fitted models in a `models` variable

Note how we use the `sklearn.base.clone` function to construct an unfitted cloned instance of the same model.

In [ ]:
from sklearn.base import clone

models = []
fold_index = np.zeros_like(y, dtype="int")
yscores = np.zeros_like(y, dtype="float")

for ifold, (itrain, itest) in enumerate(cv_strat.split(X, y)):
    Xtrain = X.iloc[itrain]
    ytrain = y[itrain]
    Xtest = X.iloc[itest]

    clf = clone(model_rf_cv)
    clf.fit(Xtrain, ytrain)

    yscores[itest] = clf.predict_proba(Xtest).T[1]
    fold_index[itest] = ifold
    models.append(clf)


We can now enrich the original dataframe with the scores and fold indices:

In [ ]:
df_attr_scored = df_attr.assign(attr_score=yscore, fold_index=fold_index)
df_attr_scored.to_csv("./scored_attrition.csv", index=False)

We can also save the models. Using the `joblib` library is preferred to the `pickle` library (it does the same but more efficiently for Numpy arrays, which are used extensively in most Scikit-Learn models).

When the model files need to be moved around (on a network for example), the use of `cloudpickle` may be considered too.

In [ ]:
import joblib

joblib.dump((models, df_attr_scored), "./model_data.pkl")

## Going further

### Note on calibration

In the previous section we have used different models to score the full dataset. Most probably, the score will be used to rank employees, so we implicitly assumed that the scores returned by the different models are comparable which is not necessarily the case.

Scikit-learn [proposes some tools](https://scikit-learn.org/stable/modules/calibration.html) to help ensure the models are correctly calibrated. However, calibration may require some additional loop of validation process. This is especially true for models which do not have a `predict_proba()` method.

### Note on multi-class classification

We treated our case study as a binary classification problem. There are many cases where there are more than one class to discriminate.

Let's take the digits classification example.

In [ ]:
from sklearn.datasets import load_digits

digits_ds = load_digits()
print(digits_ds["DESCR"])

So the images we treat here look like this:

In [ ]:
from ipywidgets import widgets, interact

slider = widgets.IntSlider(min=0, max=len(digits_ds.images) - 1, value=0)


@interact(index=slider)
def plot_digit(index):
    plt.figure(figsize=(2, 2))
    plt.title(f"Digit: {digits_ds.target[index]}")
    plt.imshow(digits_ds.images[index], cmap="gray")

The images will be simply represented by there pixel values flattened into a single row.

Basically, starting from the $8 \times 8$ pixels array the indexes of the 64 elements of the flattened array are organized like:

$$\left[\matrix{0 & 1  & \dots & 7 \\
                8 & 9 & \dots & 15 \\
                \vdots & \vdots & \ddots & \vdots\\
                56 & 57 & \dots & 63
                }\right]$$

And we can check:

In [ ]:
(digits_ds.data.reshape(-1, 8, 8) == digits_ds.images).all()

We can use TSNE to see how the data points are clustered:

In [ ]:
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

tsne = Pipeline([("scaler", StandardScaler()), ("tsne", TSNE(n_components=2))])

Xtr = tsne.fit_transform(digits_ds.data)

_ = sns.scatterplot(x=Xtr.T[0], y=Xtr.T[1], hue=digits_ds.target.astype(str))

We can see that the data points are rather well clustered.

#### Naturally multi-class algorithms

There are a few algorithms that are *naturally* multi-class, those are algorithms which are based on class counts such as:

* Tree-based algorithms : where we count the number of instances of each class falling into each tree leaf.
* Nearest neighbors classification: where we count the number of instances of each class in the neighborhood of the point.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(
    digits_ds.data, digits_ds.target, random_state=1
)

clf = RandomForestClassifier(n_estimators=100, random_state=1)
clf.fit(Xtrain, ytrain)
ypred = clf.predict(Xtest)

print(classification_report(ytest, ypred))

Let's have a look at the probabilities attributed to each class:

In [ ]:
yproba = clf.predict_proba(Xtest)
yproba

So we see that for each instance (rows) we have the probability for each class (columns). Let's check that those probabilities sum to 1:

In [ ]:
np.allclose(yproba.sum(1), 1)

From the score above we known that we make few errors. Let's identify the misclassified samples:

In [ ]:
np.where(ytest != ypred)[0]

Let's try to plot those probabilities:

In [ ]:
def plot_class_proba(index):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.bar(range(10), yproba[index])
    plt.title(f"Sample target: {ytest[index]}")
    plt.xticks(range(10))
    plt.ylim(0, 1)
    plt.ylabel("Class probability")
    plt.xlabel("Class")
    plt.subplot(1, 2, 2)
    plt.imshow(Xtest[index].reshape(8, 8))


plot_class_proba(123)

Let's make this interactive:

In [ ]:
from ipywidgets import widgets, interact

print("Missclassified:", np.where(ytest != ypred)[0])

slider = widgets.IntSlider(min=0, max=len(Xtest) - 1, value=0)
_ = interact(plot_class_proba, index=slider, continuous_update=False)

#### One versus rest strategy

When we need to use an algorithm which is binary by construction, it is possible to combine several binary classifiers. The most common strategy is the **one vs rest** strategy (a.k.a. **one vs all**)

In this strategy we build `n_classes` classifiers. Each classifier is trained using 1 class as positive and all other classes as negative.

This strategy has at least 2 main advantages:

* It requires only one classifier per class
* It is relatively easy to interpret since each classifier can be examined

The result of the prediction corresponds to the class for which the associated classifier gives the **highest positive class probability** or the **highest value of the decision function**. This assumes that the classifiers are **calibrated**.

Scikit-learn has tools to transform any binary classifier into a multi-class classifier:

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(
    digits_ds.data, digits_ds.target, stratify=digits_ds.target, random_state=1
)

clf = OneVsRestClassifier(LogisticRegression(max_iter=1000))
clf.fit(Xtrain, ytrain)

ypred = clf.predict(Xtest)
print(classification_report(ytest, ypred))

Actually most scikit-learn classifiers already implement some multi-class strategies by default:

In [ ]:
clf = LogisticRegression(max_iter=5000)
clf.fit(Xtrain, ytrain)

ypred = clf.predict(Xtest)
print(classification_report(ytest, ypred))

#### One Vs One Strategy

The one vs one strategy consists in building a model for each pair of classes. This means that for `n_classes` we build `n_classes * (n_classes - 1) / 2` classifiers.

The prediction corresponds to a majority vote of the classifiers.

This strategy has the following disadvantages:

* More individual classifiers
* Difficult to interpret

However, it has the following advantages:

* Can be overall more computationally efficient with algorithms which do not scale well with the number of samples (e.g. SVM with kernels)
* Pairs of classes can be easier to separate

For instance, look at the example below:

In [ ]:
Xs = [np.random.randn(50, 2)]
ys = [0] * 50

n_angles = 6

for i, angle in enumerate(np.arange(0, 2 * np.pi, 2 * np.pi / n_angles)):
    dx = 5.0 * np.cos(angle)
    dy = 5.0 * np.sin(angle)
    Xi = np.random.randn(20, 2) + (dx, dy)
    Xs.append(Xi)
    ys.append([i + 1] * 20)

X = np.vstack(Xs)
y = np.hstack(ys)

_ = sns.scatterplot(x=X.T[0], y=X.T[1], hue=y.astype(str))

We have seven classes of points in 2d. The class '0' is surrounded by the other classes which are arranged on a circle around it.

Clearly in a OvR approach, a linear classifier will not be able to separate the classes while in a OvO approach all pairs of classes are linearly separable.


Of course there is an estimator for this in Scikit-learn:

In [ ]:
from sklearn.multiclass import OneVsOneClassifier

clf = OneVsOneClassifier(LogisticRegression(max_iter=1000))
clf.fit(Xtrain, ytrain)

ypred = clf.predict(Xtest)
print(classification_report(ytest, ypred))

Knowing that OvR is usually the approach implemented by default for models that natively support multi-class,it might be interesting is some cases to test for the OvO too...

#### Performance measures for multi-class problems

As already noted, the ideal performance measure should reflect the real cost of a misclassification. Since it is often difficult to estimate this cost we often rely on common performance measures.

Most of the measures we have seen earlier are defined for binary classifiers only. Extended them to multi-class is more or less easy.

In the binary case, count based measures (accuracy, precision, recall, ...) can be expressed as the elements of the confusion matrix.

It is easy to extend the notion of confusion matrix to the multi-class case:

In [ ]:
df_conf = pd.DataFrame(data={"ytrue": ytest, "ypred": ypred}).pivot_table(
    index="ytrue", columns="ypred", aggfunc="size", fill_value=0
)

_ = sns.heatmap(df_conf, cmap="BuPu", annot=True, fmt="0.0f")

In the multi-class case, the notion of Positive and Negative cannot be used so the definitions of the various measures is not trivial.

For instance, the notion of **true positive** and **true negative** becomes irrelevant and should be replaced by the notion of **correct classification**. The notion of **false positive** or **false negative** has no more sense since all combinations of classes need to be taken into account.

So the performance measures will usually be expressed as a **weighted combination of the elements for the confusion matrix $C$**. We will define the element $C_{ij}$ as the number of instances of class $i$ which have been predicted as $j$.

Most often with the form:

$$Perf = \frac{\sum_{i, j}w_{ij}C_{ij}}{\sum_{kl}w'_{kl}C_{kl}}$$

Ideally, the choice of the coefficients should be adapted to take into account the cost of a specific type of mis-classification.

Hence the **the accuracy** can be redefined as:

$$Acc = \frac{\sum_{i=1}^K C_{ii}}{\sum_{k,l} C_{kl}} = \frac{\sum_{i=1}^K C_{ii}}{N_{tot}}$$

The notion of precision and recall cannot be easily adapted. Most often they are computed in a OvR approach independently for each class. And the values are averaged with or without weighting by the class support (the proportion of the class).

This is what the `classification_report()` function does:

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(ytest, ypred))

The same approach is usually used to extend the measured based on curves such as ROC or Average Precision. For the Average Precision the measure is called **Mean Average Precision**.

#### Multi-class vs multi-label

One should not confuse multi-class and multi-label problems. In **multi-class problems** the classes are **exclusive**. While in **multi-label problems**, the different classes can occur **simultaneously**.

For instance the task of determining what is the main subject of images like:

![Image-classification](./figures/image-classification.webp)

is an example of **multi-class classification**.


In the image below, if the aim is to retrieve:

* cat
* couch
* remote

![Image-multi-labels](./figures/Cat_on_couch_with_remote.jpg)

then we can consider it as a **multi-label** problem.

### Bayesian optimization

We have seen two hyper-parameters exploration strategies:

* Grid search: where we brute-force all the combinations
* Randomized Search: where we randomly sample parameter combinations

Usually the performance measure seen as a function of the hyper-parameters have some regularities. There zones the hyper-parameter spaces which are consistently associated with better performances.

So ideally a one would like to:

* Explore more precisely zones of the hyper-parameter space which have been detected as promising
* Explore zones which have not been explored yet to find potential better sets of parameters

In Bayesian Optimization, we build a "proxy" (a.k.a surrogate) function which approximates the score function (w.r.t hyper parameters). The approximate model usually provides not only a point estimate of the score value but a complete predictive distribution. This allows to compute some interesting quantities like the **Expected improvement** which can be used to choose the next hyper-parameter combination.

Among libraries that can be used we can cite:

* [`scikit-optimize`](https://scikit-optimize.github.io/stable/)
* [`hyperopt`](https://github.com/hyperopt/hyperopt)
* [`optuna`](https://optuna.readthedocs.io)